# Opdracht 4.3: de Goldberg-Variationen

## Introductie

De [Goldberg-variationen (BWV 988)](https://de.wikipedia.org/wiki/Goldberg-Variationen) is een muziekstuk dat rondom 1741 door [Johann Sebastian Bach](https://de.wikipedia.org/wiki/Johann_Sebastian_Bach) geschreven is. Het betreft een Aria met een dertigtal variaties hier op. De naam van het werk is anekdotisch: hierin wordt gesuggereert dat Bach het werk schreef voor [Johann Gottlieb Goldberg](https://de.wikipedia.org/wiki/Johann_Gottlieb_Goldberg), de muziekleraar van [graaf Hermann Carl von Keyserlinkgk](https://de.wikipedia.org/wiki/Hermann_Carl_von_Keyserlingk), opdat deze zijn slapeloze nachten wat prettiger door kon komen. Hoewel het werk oorspronkelijk voor clavicimbel geschreven is, wordt het ook vaak op de piano uitgevoerd ([hier](https://www.youtube.com/watch?v=p4yAB37wG5s) bijvoorbeeld door niemand minder dan [Glenn Gould](https://nl.wikipedia.org/wiki/Glenn_Gould)).

In deze opgave gaan we met een RNN een eenendertigste variatie aan dit werk toevoegen. Er zijn natuurlijk verschillende manieren om dit probleem te adresseren. We zouden bijvoorbeeld de partituur kunnen scannen en op basis van beeldherkenning en -generatie hier een tweetal nieuwe pagina's aan toevoegen.

Hier kiezen we er evenwel voor om de bladmuziek om te zetten in een tekstuele representatie. Dan wordt het maken van een nieuwe variatie feitelijk een kwestie van tekstgeneratie, waar LSTM's en GRU's goed in zijn. Er zijn gelukkig verschillende technieken om dit te doen (zie met name [Marinescu, 2019](https://doi.org/10.1016/j.procs.2019.09.166)), maar we gebruiken hier de midi-bestanden [van Dave's J.S. Bach page](http://www.jsbach.net/midi/midi_goldbergvariations.html) die we met behulp van [py_midicsv](https://pypi.org/project/py_midicsv/) hebben omgezet in csv-bestanden: één bestand voor elke variatie. Je vindt deze bestanden in [de zip `data/bwv988.zip`](data/bwv988.zip).

In de cellen hieronder staan de verschillende onderdelen toegelicht. In grote lijnen ziet het stappenplan er als volgt uit:

0. importeer de noodzakelijke bibliotheken (om uiteindelijk de muziek te kunnen horen moet je waarschijnlijk [py_midicsv](https://pypi.org/project/py_midicsv/) nog even pip installen)
1. laad de databestanden in één grote lijst
2. preprocess de data
3. maak de vectoren `x` en de `y` en one-hot-encode deze
4. maak en train het model
5. maak een methode die op basis van een `seed` een nieuwe sequentie genereert af
6. zet die nieuwe sequentie om in een midi-bestand en geniet van de nieuwe variatie

## Stap 0: importeer de noodzakelijke bibliotheken en afhankelijkheden

Run de onderstaande cel

In [11]:
import os
import random
from pathlib import Path 
from keras.utils import to_categorical

import numpy as np
import pandas as pd
from utils import *

## Stap 1: imporeer de data

Pak de zip met de data uit in de directory `data` (of ergens anders wat voor je werkt). Dit zijn csv-versies van de midi-bestanden die we van [Dave's J.S. Bach page](http://www.jsbach.net/midi/midi_goldbergvariations.html) hebben afgehaald – bekijk de individuele bestanden om een beeld te krijgen van hoe die dingen eruit zien. Loop door al deze bestanden en laad de inhoud hiervan in de lijst `data`. Je kunt gebruik maken van [`pathlib.Path`](https://docs.python.org/3/library/pathlib.html).

In [36]:
data = []
# YOUR CODE HERE
pathlist = Path('data/bwv988').glob('*.txt')
for path in pathlist:
    data.append(path.open().read())
    
print(data)

['yW yW yW yW yW yW yW yW yW yW yW yW yW yW yW yW yCW yCW yCW yCW yCW yCW yCW yCW yCW yCW yCW yCW yCW yCW yCW yCW yCFY yCFY yCFW yCFW yCFY yCFY yCFY yCFY yCFY yCFY yCFY yCFY yCF! yCF! yCF! yCF! xY xY xY xY xY xY xY xY xW xW xW xW xV xV xV xV xAT xAT xAT xAT xAT xAT xAT xAT xAR xAR xAR xAR xAR xAR xAR xAR xAFR xAFR xAFR xAFR xAFR xAFR xAFR xAFR xAFR xAFR xAFR xAFR xAFR xAFR xAFR xAFR vK vK vJ vJ vK vK vK vK vK vK vK vK vK vK vK vK vyM vyM vyK vyK vyJ vyJ vyK vyK vyM vyM vyK vyK vyM vyM vyK vyK vyEM vyEK vyEK vyEM vyEK vyEM vyEM vyEK vyEJ vyEJ vyEJ vyEJ vyEK vyEK vyEK vyEK tM tM tK tK tJ tJ tJ tJ tK tK tJ tJ tH tH tH tH txH txH txH txH txH txH txH txH txF txF txF txF txF txF txF txF txAF txAF txAF txAF txAF txAF txAF txAF rxAF rxAF rxAF rxAF rxAF rxAF rxAF rxAF qR qR qR qR qR qR qR qR qR qR qR qR qR qR qR qR qtR qtR qtR qtR qtR qtR qtR qtR qtR qtR qtR qtR qtR qtR qtR qtR qtyT qtyT qtyR qtyR qtyT qtyT qtyT qtyT qtyT qtyT qtyT qtyT qtyU qtyU qtyU qtyU rT rT rT rT rT rT rT rT rR rR rR rR rP

In [13]:
len(data) # zou 31 moeten zijn (de aria en dertig variaties)

31

## Stap 2: preprocessing

Het preprocessen van de data kent op zich weer een aantal stappen. Omdat we uiteindelijk een karakter-gebaseerde sequentie-generator gaan maken, is het van belang om een lijstje te hebben van alle *unieke* karakters die in de data voorkomen. Verder moeten we die karakters om kunnen zetten in getallen, omdat we die getallen aan het model gaan voeren; en om uiteindelijk de tonen (karakters) weer terug te kunnen krijgen, moeten we ook die getallen weer om kunnen zetten in de corresponderende karakters.

In [14]:
# YOUR CODE HERE
# vervang None door de juiste code

chars = None
char_to_idx = None
idx_to_char = None

dataSplit = set()
for string in data:
    splitString = list(string)
    dataSplit.update(splitString)

chars = list(dataSplit)

char_to_idx = {}
for i in range(len(chars)):
    char_to_idx.update({chars[i]: i})
    
print(char_to_idx)

idx_to_char = dict([(value, key) for key, value in char_to_idx.items()])

print(idx_to_char)


{'x': 0, 'z': 1, 'R': 2, 'V': 3, 'h': 4, '$': 5, 'U': 6, 's': 7, 'S': 8, 'j': 9, 'g': 10, 'M': 11, 'O': 12, 'D': 13, 'y': 14, 'o': 15, 'F': 16, 'k': 17, 'Q': 18, 'J': 19, ' ': 20, 'n': 21, 'Z': 22, 'v': 23, 't': 24, 'K': 25, 'i': 26, 'B': 27, 'L': 28, 'l': 29, 'f': 30, 'P': 31, 'q': 32, 'm': 33, 'H': 34, 'r': 35, 'u': 36, 'E': 37, '%': 38, 'I': 39, 'e': 40, '#': 41, '!': 42, 'T': 43, 'N': 44, 'W': 45, 'G': 46, 'c': 47, 'p': 48, 'A': 49, 'X': 50, 'w': 51, 'C': 52, 'a': 53, 'Y': 54}
{0: 'x', 1: 'z', 2: 'R', 3: 'V', 4: 'h', 5: '$', 6: 'U', 7: 's', 8: 'S', 9: 'j', 10: 'g', 11: 'M', 12: 'O', 13: 'D', 14: 'y', 15: 'o', 16: 'F', 17: 'k', 18: 'Q', 19: 'J', 20: ' ', 21: 'n', 22: 'Z', 23: 'v', 24: 't', 25: 'K', 26: 'i', 27: 'B', 28: 'L', 29: 'l', 30: 'f', 31: 'P', 32: 'q', 33: 'm', 34: 'H', 35: 'r', 36: 'u', 37: 'E', 38: '%', 39: 'I', 40: 'e', 41: '#', 42: '!', 43: 'T', 44: 'N', 45: 'W', 46: 'G', 47: 'c', 48: 'p', 49: 'A', 50: 'X', 51: 'w', 52: 'C', 53: 'a', 54: 'Y'}


In [15]:
len(chars) # zou 55 moeten opleveren: wat representeert dit aantal?

55

Nu we karakters kunnen vertalen in getallen, kunnen we de data omzetten in de corresponderende indexen.

In [16]:
# YOUR CODE HERE:
# vervang None door de juiste code

encoded_sequence = []

for string in data:
    fucks = list(string)
    for fuck in fucks:
        encoded_sequence.append(char_to_idx[fuck])   


In [17]:
len(encoded_sequence) # zou 309378 moeten opleveren

309378

In de cel hieronder zetten we twee waarden die we verderop nodig hebben. Je kunt eventueel experimenteren met verschillende lenges van `sequence_length`.

In [18]:

vocab_size = len(chars)
sequence_length = 32  


## Stap 3: matrixen, vectoren en one hot encoding

Maak nu de data-matrix `X` en de target-vector `y` op dezelfde manier als we hebben gedaan in opgave 4.2, bij die `n-grams`. Zie eventueel hetzelfde plaatje hieronder. Net als bij opgave 4.2 kun je hier prima standaard python-lijsten voor gebruiken; we zetten ze later om in numpy-datatypen.

[n-grams](imgs/n-gram.png)

In [19]:
X = []
y = []

for i in range(len(encoded_sequence) - sequence_length):
    X.append(encoded_sequence[i:i + sequence_length])
    y.append(encoded_sequence[i + sequence_length])

X = np.array(X)
y = np.array(y)

Zet nu `X` en `y` om in one-hot encoded matrices met een breedte van `vocab_size`. Maak gebruik van [`tf.keras.utils.to_categorical`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) (of je eigen uitwerking uit week 2).

In [20]:
X = to_categorical(X, num_classes=vocab_size)
y = to_categorical(y, num_classes=vocab_size)

print(X.shape)
print(y.shape)

(309346, 32, 55)
(309346, 55)


## Stap 4: maken en trainen van het model

Als model maken we gebruik van een recurrent LSTM netwerk – gebruik de relevante klassen uit [`Keras.layers`](https://keras.io/api/layers/). Gebruik een [`Input`]() als eerste laag (bedenk zelf wat de correcte waarden voor de `shape`-parameter zijn). Voeg minimaal *twee* [`LSTM-layers`](https://keras.io/api/layers/recurrent_layers/lstm/) toe en zorg ervoor dat de laatste *state* hiervan aan de output wordt meegegeven - bestudeer de documentatie om te zien hoe je dat moet doen. Voeg als laatste laag een [`Dense`](https://keras.io/api/layers/core_layers/dense/) toe met een grootte van `vocab_size` en 'softmax' als activatie-functie. Vergewis je ervan dat je begrijpt waarom je juist deze waarden moet gebruiken.

In [25]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input

#vervang None door de juiste code
model = Sequential([
    Input(shape=(sequence_length, vocab_size)),
    LSTM(128, return_sequences=True),
    LSTM(128),
    Dense(vocab_size, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')



In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 32, 128)           94208     
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 55)                7095      
                                                                 
Total params: 232,887
Trainable params: 232,887
Non-trainable params: 0
_________________________________________________________________


Gebruik nu de methode `fit` om het model te trainen. Kies relevante opties voor `epochs` en voor `batch_size`. __Let op:__ het trainen van het netwerk kan even duren. Op mijn macbook air uit 2020 met een M1-processor duurde één epoch een kleine vier minuten ⏳.

In [27]:
model.fit(X,y,epochs=1, batch_size=128)

2417/2417 [==============================] - 315s 128ms/step - loss: 1.1101


# Stap 5: Nieuwe karakters voorspellen

Maak nu de methode `sample` hieronder af. Deze methode krijg een `seed` mee die in het getrainde model gestopt wordt. Dit model voorspelt op basis van deze `seed` de verdeling van de waarschijnlijkheid van alle karakters uit `vocab`. Het stappenplan voor deze methode is als volgt:

1. one-hot encode de geëncodeerde `seed`
2. gebruik `model.predict` om de index van het volgende karakter te voorspellen
3. maak gebruik van `utils.add_temperatature` om het geheel iets stochastischer (en dus interessanter) te maken
4. kies één van de indexen uit de voorspelling, op basis van de distributie die het model heeft gegeven (bestudeer de documentatie van [`numpy.random.choice`]() om te zien hoe je dit eenvoudig kunt doen.
5. zet deze gekozen index om in het corresponderende karakter uit `vocab` - maak gebruik van de dictionary `idx_to_char` die je hierboven hebt gemaakt.
6. voeg dit karakter toe aan de gegenereerde tekst
7. voeg de gekozen index toe aan `encoded_seed` en verwijder, om de grootte constant te houden, het eerste karakter.



In [34]:
def sample(model, seed_text, num_generate, temperature=1.0):
    generated_text = seed_text
    encoded_seed = [char_to_idx[char] for char in seed_text]

    for _ in range(num_generate):
        seed = to_categorical(encoded_seed, num_classes=vocab_size)
        prediction = model.predict(np.array([seed]))[0]
        prediction = add_temperature(prediction, temperature)
        idx = np.random.choice(range(vocab_size), p=prediction)
        generated_text += idx_to_char[idx]
        encoded_seed.append(idx)
        encoded_seed = encoded_seed

    return generated_text



## Stap 6: een nieuwe variatie

Gebruik nu de eerste paar maten van een variatie, of van de aria zelf, om met behulp van `sample` een nieuwe variatie te maken. Kies een relevante waarde voor `num_generate` zodat je voor een paar minuten karakters terugkrijgt. __Let op:__ Het genereren van een fatsoenlijk stuk kan ook wel even duren. 

Sla het gegenereerde bestand op en gebuik de scripts `deprocess.py` en `csvtomidi.py` om het uiteindelijke midi-bestand te creëren. Zie het voorbeeld hieronder (ervan uitgaande dat het gegenereerde bestand is opgeslagen onder de naam `result.txt`):

```shell
python deprocess.py -i result.txt -o tmp.txt --tempo 250000
python csvtomidi.py -i tmp.txt -o variatie31.mid
```

Je kunt midi afspelen met VLC, of je kunt het online omzetten naar bijvoorbeeld mp3. Speel het af en geniet van je eigen bach-creatie 🎼🎶.


In [35]:
# Choose an interesting string as seed
seed_text = 'Da '

# Number of characters to generate
num_generate = 1000

# Generate new variation
new_variation = sample(model, seed_text, num_generate, temperature=1.0)

# Save the generated text to a file
with open('result.txt', 'w') as f:
    f.write(new_variation)

# Print the generated text
print(new_variation)


1/1 [==============================] - 0s 263ms/step
Da aMzCwyyFKjZm!qmnOtm!mtctBt mptJ ppJ p#J ppY ppp# pppttwpq#tp#cLABPszfcsqP pvR p yN pyZ pBN pJ pmBIpqqfjCZkr$uUpwyiHqYoDYorpKY ppBY ppI B pBBBUBBUZB !UvNBHfOokUAUOxRGqVNBU BUV BUT BRU BDUZ%BAU!wyXiokhya$JGuiknBBHzIy tJO tHO tHO yHX yIYXX$#nXXWpW$RUUlnLKtNlksoTO oOU oTU qOU zUU LTUTLUhOUfzRgPMkElAoInpPqXDEU zRT zQU zIU zI ID! LhXUX%D$a#S$BDnDnBXr%CzCw!TAPCAXFPP APPP APPP PPzYP ezY YPRnfy$iFiPXpjrhvIKwooyIL ooII ooqooo oooooqooooKoooooo ooo oooooovoooooqooIoooooqooooooooovooR ovoooovKovooooooooQooooJoooooo  tototooErooooolooTFo olooomoooohoqooooscco h%loootsvghzoomoMgo voootopsJoqooVcvo xmoQRKcJtvvcoqRoovosoopoooHoorocl oqqlooV mprsootoomoooxpon oosoqeq mmsoIpTBrVa!Ltrv$xRqqYrqrDY qrrCG qrqrqrvrIXqqryqqYqqrjostqLXymIDwyGKnonHHH HHHHHHjHHGHHHHHHQHHHHQHHHHHH HGKHJAHJHIYoz#DiZazLuaMnVgjGjnnntsemmnnpn pnpzmmmoz lpozIpp$ooRywHqqWqplXrnsLfUsRqptSR!pppp#T ppT pU pTT pUoRs%ppaBZjGpaaYnGysvmtsmtfmyO myyO myy qmmCqqqcq pzxmfjWsyJeqQAkoyGRettyB

## Verdere stappen

Misschien vind je de nieuwe variatie nog niet zo heel fraai – misschien wel een beetje *John Cage meets JS Bach*. Je kunt proberen het model wat uit te breiden (bijvoorbeeld door een [`GRU`]() laag toe te voegen), de `sequence_length` te vergroten of meer of juist minder karakters te genereren. Experimenteer hier wat mee en bekijk (of beluister eigenlijk) wat het schoonste resultaat oplevert. Of je kunt natuurlijk meer bestanden van Dave's JS Bach Page halen en kijken of je een heel nieuw werk kunt genereren.

**Credits**

Deze opgave is gemaakt op basis van [het genoemde artikel van Alexandru-Ion Marinescu](https://doi.org/10.1016/j.procs.2019.09.166) en een vergelijkbaar experiment van [Tobias van der Werf](https://github.com/tobiasvanderwerff).